In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [2]:
data = pd.read_csv("master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features

# Dropped all categorical data 

data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 'polarity_category', 'overall_content', 'polarity_category_Neutral',
 'polarity_category_Positive'], axis=1)
#'Topic 1 Probability', 'Topic 2 Probability', 'Topic 3 Probbility' , 'Topic 4 Probability' ,'Topic 5 Probability',
#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity_category_Neutral' , 'polarity_category_Positive'

In [3]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [4]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'text_average_word_length',
 'title_average_word_length',
 'text_punctuation_count',
 'title_punctuation_count',
 'text_stopwords_count',
 'title_stopwords_count',
 'flesch_readability',
 'subjectivity',
 'polarity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability']

In [5]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,5.138347,5.592058,53.119147,1.715635,155.440685,1.136789,0.006674,0.402753,0.056997,0.190953,0.324903,0.125233,0.185360,0.173552
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.875575,1.430398,63.542364,1.452448,123.275458,1.295738,1.003826,0.124806,0.105170,0.281023,0.312565,0.210573,0.258829,0.234861
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.250000,3.000000,0.000000,0.000000,0.000000,0.000000,-83.616811,0.000000,-1.000000,0.000376,0.000246,0.000120,0.000303,0.000268
25%,0.000000,220.000000,10.000000,7.000000,1.000000,4.866522,5.000000,26.000000,1.000000,80.000000,0.000000,-0.169506,0.337127,0.000000,0.002949,0.010969,0.001305,0.003113,0.003024
50%,0.000000,376.000000,11.000000,13.000000,1.000000,5.075727,5.500000,45.000000,2.000000,142.000000,1.000000,0.050549,0.405745,0.054757,0.030471,0.231605,0.006765,0.056382,0.056373
75%,1.000000,522.000000,14.000000,20.000000,1.000000,5.272987,6.066667,67.000000,3.000000,201.000000,2.000000,0.267479,0.475000,0.108144,0.282926,0.584998,0.157612,0.272065,0.268291
max,1.000000,8436.000000,45.000000,321.000000,4.000000,149.000000,149.000000,7295.000000,26.000000,3017.000000,15.000000,2.614284,1.000000,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767


In [6]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 4222)


x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = undersample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()

0    10477
1    10477
Name: class, dtype: int64

In [7]:
x_train

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
0,350,12,9,1,4.884393,5.000000,43,1,139,2,-0.118632,0.455827,0.112187,0.663325,0.250534,0.001214,0.083110,0.001817
1,409,9,17,1,5.024570,4.555556,44,0,158,2,0.057910,0.416667,0.069369,0.418191,0.577424,0.001063,0.001737,0.001585
2,824,10,31,1,5.241169,5.500000,96,2,284,1,-0.202569,0.396190,-0.015337,0.001535,0.545127,0.012585,0.401411,0.039342
3,82,10,2,1,5.144737,5.100000,7,3,28,2,-0.199150,0.248148,0.166667,0.968729,0.008208,0.005579,0.009101,0.008383
4,803,9,32,1,4.630155,6.444444,77,1,329,0,0.461543,0.296639,0.034829,0.405976,0.472325,0.000872,0.119526,0.001302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,565,16,10,1,4.742857,5.533333,46,1,256,0,-0.267928,0.368203,0.122754,0.002057,0.364870,0.000975,0.630648,0.001450
20950,1809,32,144,1,4.418500,4.875000,307,3,772,9,0.852104,0.467778,-0.001175,0.015731,0.304634,0.000364,0.000592,0.678679
20951,418,24,15,2,4.597087,4.500000,37,7,190,7,0.432620,0.487291,0.011708,0.002322,0.914441,0.062097,0.019515,0.001625
20952,677,14,25,1,4.559398,5.538462,86,2,281,0,0.490826,0.482386,0.237027,0.001734,0.258729,0.274193,0.419005,0.046338


In [8]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

scaler = StandardScaler()

# Since polarity_category_Neutral and polarity_category_Positive are already from 0 to 1 scale, we will not have to further normalize it.
cols = data.iloc[:,0:-1].columns

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
x_train[cols] = scaler.fit_transform(x_train[cols]) 

x_test[cols] = scaler.transform(x_test[cols])
x_validation[cols] = scaler.transform(x_validation[cols])



In [9]:
x_train.describe()

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04
mean,1.462036e-16,1.810796e-14,-8.632601e-16,-6.958657e-15,-5.413622e-17,6.121221e-16,3.172009e-16,-7.545095e-15,-7.198700e-16,-6.038444e-14,-5.360108e-17,-8.193418e-17,2.058369e-16,1.928198e-15,-8.150343e-16,-2.527726e-16,-3.877727e-16,-5.443108e-16
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00
min,-1.260984e+00,-2.927148e+00,-1.176626e+00,-2.601708e-01,-1.583314e+00,-1.633374e+00,-7.874532e-01,-1.186685e+00,-1.251398e+00,-8.158820e-01,-8.141493e+01,-3.257600e+00,-9.845301e+00,-6.517193e-01,-1.081183e+00,-5.961659e-01,-7.035506e-01,-7.544376e-01
25%,-5.755481e-01,-6.191153e-01,-6.723661e-01,-2.601708e-01,-1.495724e-01,-4.082828e-01,-4.037841e-01,-4.995190e-01,-5.947834e-01,-8.158820e-01,-1.882408e-01,-5.258823e-01,-5.350918e-01,-6.425415e-01,-9.946847e-01,-5.907836e-01,-6.929081e-01,-7.420030e-01
50%,-1.096360e-01,-1.062190e-01,-1.681066e-01,-2.601708e-01,-3.416247e-02,-5.825674e-02,-1.234105e-01,1.876476e-01,-1.063261e-01,-5.280436e-02,4.378139e-02,2.333645e-02,-1.957386e-02,-5.669655e-01,-2.520426e-01,-5.585622e-01,-4.975834e-01,-4.854789e-01
75%,3.324815e-01,6.631254e-01,4.201962e-01,-2.601708e-01,7.860789e-02,2.917693e-01,2.159890e-01,8.748141e-01,3.581086e-01,7.102733e-01,2.739940e-01,5.770460e-01,4.815962e-01,2.538622e-01,8.500241e-01,1.600327e-01,3.103567e-01,4.280724e-01
max,2.463666e+01,8.356569e+00,2.571722e+01,1.038752e+01,7.705558e+01,9.759900e+01,1.068612e+02,1.186948e+01,2.290722e+01,1.063028e+01,2.428422e+00,4.731028e+00,8.775117e+00,2.985253e+00,2.094804e+00,4.169520e+00,3.237046e+00,3.482659e+00


Linear SVM

In [10]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=4222)
linearSVC.fit(x_train, np.ravel(y_train))
y_predval_linearSVC = linearSVC.predict(x_validation)

#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_linearSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_linearSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_linearSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_linearSVC))
print("-------------------------------")


#test metrics
y_pred_linearSVC = linearSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))


Performance on Validation set:
Accuracy: 0.9645628556647697
Precision: 0.9679673849737915
Recall: 0.9529816513761468
F1_score: 0.9604160647211788
-------------------------------
Performance on  Test set:
Accuracy: 0.9583548887739265
Precision: 0.9643170517695232
Recall: 0.9428081212467829
F1_score: 0.9534412955465587


In [11]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state = 4222), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 100, 'gamma': 1}


In [12]:
finalSVC = SVC( kernel= 'linear', C= 100, gamma= 1, random_state = 4222)
finalSVC.fit(x_train, np.ravel(y_train))
y_predval_finalSVC = finalSVC.predict(x_validation)


#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_finalSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_finalSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_finalSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_finalSVC))
print("-------------------------------")


#test metrics
y_predtest_linearSVC = finalSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_predtest_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_predtest_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_predtest_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_predtest_linearSVC))


Performance on Validation set:
Accuracy: 0.9645628556647697
Precision: 0.9679673849737915
Recall: 0.9529816513761468
F1_score: 0.9604160647211788
-------------------------------
Performance on  Test set:
Accuracy: 0.9592602172788411
Precision: 0.9638483965014577
Recall: 0.9453817557906777
F1_score: 0.9545257687310524
